<a href="https://colab.research.google.com/github/yl2883/CornellCSWiki/blob/master/1_latent_class_analysis_stepmix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generalized Mixture Models with StepMix

The full StepMix Documentation is available [here](https://stepmix.readthedocs.io/en/latest/index.html).


## Introduction


In this tutorial, we show how to use the StepMix package for fitting mixtures of various data types. Informally, mixture models assume that an unobserved discrete random variable $Z$ (the latent class, group or cluster) can explain some observed features $X$, as depicted in the following graphical model :

<img src='https://drive.google.com/uc?export=download&id=18ESfAhLsN-wUboG3gvj_6P-SC3SkUxhR' width=150px>

Members of the same latent class will exhibit similar patterns in the observed data $X$. In this tutorial, we will show how $X$ can be continuous, categorical or a combination of both. Missing values are also discussed.

We will use the well-known [Iris Dataset](#https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). The dataset includes 4 continuous measurements of petals and sepals and is typically used to classify samples into one of 3 iris flower types.

Here, we instead aim to identify 3 groups using only the measurements. We can then compare our latent classes with ground truth flower types to assess clustering quality.

This tutorial has the following sections:


1. [Install and Import StepMix](#setup)  
2. [Setup the Iris Dataset](#data)
3. [Gaussian Mixtures (Continuous Features)](#continuous)
4. [Binary Mixtures (LCA)](#binary)
5. [Categorical Mixtures (LCA)](#categorical)
6. [Mixed-Type Mixtures](#mixed)
7. [Mixtures with Missing Values](#nan)


<a name="setup"></a>
## Install and Import StepMix

StepMix can be installed with ```pip```. We can then import the main StepMix object as well as other packages for the data and other utilities.

In [ ]:
%%capture
!pip install stepmix

In [ ]:
# Python imports
from stepmix.stepmix import StepMix
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
from sklearn.metrics import rand_score

<a name="data"></a>
## Setup the Iris Dataset

We load the Iris dataset in a ```DataFrame```. We will save most results to ```df```.

In [ ]:
# Load dataset
df, target = load_iris(return_X_y=True, as_frame=True)

# Save actual flower type name
df['iris_flower_type'] = target.map({0:'setosa', 1:'versicolor', 2:'virginica'})

<a name="continuous"></a>
## Gaussian Mixtures (Continuous Features)

We first pick and visualize the 4 input measurements in the data.

In [ ]:
# Pick continuous features
continuous_features = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
                 'petal width (cm)']
continuous_data = df[continuous_features]

continuous_data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


We then create the StepMix model and specify 3 latent classes with ```n_components=3``` and continuous measurements with ```measurement="continuous"```. All model options are described under the ```measurement``` argument in the [documentation](https://stepmix.readthedocs.io/en/latest/api.html#stepmix.stepmix.StepMix).

More formally, ```continuous``` is an alias for a Gaussian mixture model with diagonal covariance. We estimate one mean and one variance parameter per feature.

```verbose=1``` gives us a detailed output and ```random_state``` ensures reproducible results.

In [ ]:
# Gaussian mixture model
model = StepMix(n_components=3, measurement="continuous", verbose=1, random_state=123)

# Fit to data
model.fit(continuous_data)

# Save class membership predictions to df
df['continuous_pred'] = model.predict(continuous_data)

Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:00<00:00, 11.47it/s, max_LL=-307, max_avg_LL=-2.05]

MODEL REPORT
    Measurement model parameters
          model_name                    gaussian_diag                
          class_no                                  0       1       2
          param       variable                                       
          covariances petal length (cm)        0.0296  0.2254  0.3269
                      petal width (cm)         0.0109  0.0348  0.0851
                      sepal length (cm)        0.1218  0.2288  0.3246
                      sepal width (cm)         0.1408  0.0870  0.0827
          means       petal length (cm)        1.4620  4.2225  5.4829
                      petal width (cm)         0.2460  1.3044  1.9896
                      sepal length (cm)        5.0060  5.8346  6.6227
                      sepal width (cm)         3.4280  2.7001  3.0171
    Class weights
        Class 1 : 0.33
        Class 2 : 0.31
        Class 3 : 0.36
    Fit for 3 latent classes
    Estimation method             : 1-step
    Number of observation

We then compare the class predictions of our Gaussian Mixture Model and the iris flower types. We find that a group is perfectly aligned with setosa. The 2 other groups mostly correspond to versicolor and virginica, with some confusion between both.

In [ ]:
pd.crosstab(df['iris_flower_type'], df['continuous_pred'])

continuous_pred,0,1,2
iris_flower_type,,,
setosa,50,0,0
versicolor,0,43,7
virginica,0,2,48


A quick way to measure the agreement of two clusterings is to use the [Rand Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.rand_score.html). The rand score is between 0 and 1, with 1 indicating perfect agreement.

In [ ]:
rand_score(df['iris_flower_type'], df['continuous_pred'])

0.9267114093959732

<a name="binary"></a>
## Binary Mixtures (LCA)

In this section, we dichotomize the petal and sepal measurements to obtain 4 binary variables, then fit a Binary Mixture.

In [ ]:
# Create binarized features based on quantiles
binary_features = []

for c in continuous_data:
  # Create new column name
  c_binary = c.replace("cm", "q=2")
  binary_features.append(c_binary)

  df[c_binary] = pd.qcut(df[c], q=2).cat.codes

# Select columns and show dataframe
binary_data = df[binary_features]

binary_data

,sepal length (q=2),sepal width (q=2),petal length (q=2),petal width (q=2)
0,0,1,0,0
1,0,0,0,0
2,0,1,0,0
3,0,1,0,0
4,0,1,0,0
...,...,...,...,...
145,1,0,1,1
146,1,0,1,1
147,1,0,1,1
148,1,1,1,1


Here we specify ```measurement="binary"``` when creating the model.

In [ ]:
# Binary mixture model
model = StepMix(n_components=3, measurement="binary", verbose=1, random_state=123)

# Fit model
model.fit(binary_data)

# Class predictions
df['binary_pred'] = model.predict(binary_data)

Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:00<00:00, 12.78it/s, max_LL=-268, max_avg_LL=-1.79]

MODEL REPORT
    Measurement model parameters
          model_name                binary                
          class_no                       0       1       2
          param variable                                  
          pis   petal length (q=2)  0.8580  0.0000  1.0000
                petal width (q=2)   0.7426  0.0000  1.0000
                sepal length (q=2)  0.5618  0.0379  1.0000
                sepal width (q=2)   0.0000  0.5890  0.4744
    Class weights
        Class 1 : 0.17
        Class 2 : 0.48
        Class 3 : 0.35
    Fit for 3 latent classes
    Estimation method             : 1-step
    Number of observations        : 150
    Number of latent classes      : 3
    Number of estimated parameters: 14
    Log-likelihood (LL)           : -267.7836
    -2LL                          : 535.5672
    Average LL                    : -1.7852
    AIC                           : 563.57
    BIC                           : 605.72
    CAIC                          : 619.72
 

The cross tabulation and the Rand Score show that our groups somewhat align with the flower types, but not as well as those obtained with Gaussian Mixture Model from the previous section.

In [ ]:
pd.crosstab(df['iris_flower_type'], df['binary_pred'])

binary_pred,0,1,2
iris_flower_type,,,
setosa,0,50,0
versicolor,9,23,18
virginica,6,0,44


In [ ]:
rand_score(df['iris_flower_type'], df['binary_pred'])

0.7276957494407159

<a name="categorical"></a>
## Categorical Mixtures (LCA)


StepMix also supports categorical variables with more than 2 categories. Categories must be integer-encoded.

Similarly to the previous section, we discretize the continuous measurements by binning the data into 3 quantiles, yielding 4 categorical features with 3 categories each.

In [ ]:
# Create categorical features based on quantiles
categorical_features = []

for c in continuous_data:
  # Create new column name
  c_categorical = c.replace("cm", "q=3")
  categorical_features.append(c_categorical)

  df[c_categorical] = pd.qcut(df[c], q=3).cat.codes

# Select columns and show dataframe
categorical_data = df[categorical_features]

categorical_data

,sepal length (q=3),sepal width (q=3),petal length (q=3),petal width (q=3)
0,0,2,0,0
1,0,1,0,0
2,0,1,0,0
3,0,1,0,0
4,0,2,0,0
...,...,...,...,...
145,2,1,2,2
146,1,0,2,2
147,2,1,2,2
148,1,2,2,2


In [ ]:
# Categorical mixture model
model = StepMix(n_components=3, measurement="categorical", verbose=1, random_state=123)

# Fit model
model.fit(categorical_data)

# Class predictions
df['categorical_pred'] = model.predict(categorical_data)

Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:00<00:00, 18.69it/s, max_LL=-412, max_avg_LL=-2.75]

MODEL REPORT
    Measurement model parameters
          model_name                 categorical                
          class_no                             0       1       2
          param variable                                        
          pis   petal length (q=3)_0        1.00  0.0000  0.0000
                petal length (q=3)_1        0.00  0.0000  0.9124
                petal length (q=3)_2        0.00  1.0000  0.0876
                petal width (q=3)_0         1.00  0.0000  0.0000
                petal width (q=3)_1         0.00  0.0141  0.8689
                petal width (q=3)_2         0.00  0.9859  0.1311
                sepal length (q=3)_0        0.90  0.0000  0.1183
                sepal length (q=3)_1        0.10  0.2310  0.7024
                sepal length (q=3)_2        0.00  0.7690  0.1793
                sepal width (q=3)_0         0.04  0.3287  0.7026
                sepal width (q=3)_1         0.30  0.4760  0.2631
                sepal width (q=3)_2         

This model performs nearly as well as the [gaussian mixture model](#continuous) tested above.

In [ ]:
pd.crosstab(df['iris_flower_type'], df['categorical_pred'])

categorical_pred,0,1,2
iris_flower_type,,,
setosa,50,0,0
versicolor,0,1,49
virginica,0,40,10


In [ ]:
rand_score(df['iris_flower_type'], df['categorical_pred'])

0.9123937360178971

<a name="mixed"></a>
## Mixed-Type Mixtures

StepMix can also combine continuous and categorical features into a single model. For illustrative purposes, we will use two continuous features, one binary feature and one categorical feature. We build a model description using the ```get_mixed_descriptor``` utility.

In [ ]:
# More complex models need a more complex description
# StepMix provides a function to quickly build mixed descriptors for DataFrames
from stepmix.utils import get_mixed_descriptor

# Unspecified variables are simply not included in mixed_data
mixed_data, mixed_descriptor = get_mixed_descriptor(
    dataframe=df,
    continuous=['sepal length (cm)', 'petal length (cm)'],
    binary=['petal width (q=2)'],
    categorical=['sepal width (q=3)']
)

mixed_data

,sepal length (cm),petal length (cm),petal width (q=2),sepal width (q=3)
0,5.1,1.4,0,2
1,4.9,1.4,0,1
2,4.7,1.3,0,1
3,4.6,1.5,0,1
4,5.0,1.4,0,2
...,...,...,...,...
145,6.7,5.2,1,1
146,6.3,5.0,1,0
147,6.5,5.2,1,1
148,6.2,5.4,1,2


The descriptor can now be passed to StepMix instead of the strings we were previously using.

In [ ]:
# Mixed-type mixture model
model = StepMix(n_components=3, measurement=mixed_descriptor, verbose=1, random_state=123)

# Fit model
model.fit(mixed_data)

# Class predictions
df['mixed_pred'] = model.predict(mixed_data)

Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:00<00:00, 11.99it/s, max_LL=-437, max_avg_LL=-2.92]

MODEL REPORT
    Measurement model parameters
          model_name                    continuous                
          class_no                               0       1       2
          param       variable                                    
          covariances petal length (cm)     0.0296  0.1645  0.3945
                      sepal length (cm)     0.1218  0.1739  0.3306
          means       petal length (cm)     1.4620  4.0062  5.2945
                      sepal length (cm)     5.0060  5.6694  6.5179




          model_name              binary             
          class_no                     0       1    2
          param variable                             
          pis   petal width (q=2)    0.0  0.0716  1.0


          model_name                categorical                
          class_no                            0       1       2
          param variable                                       
          pis   sepal width (q=3)_0        0.04  0.9282  0.3867
                sepal width (q=3)_1        0.30  0.0718  0.4701
                sepal width (q=3)_2        0.66  0.0000  0.1432


    Class weights
        Class 1 : 0.33
        Class 2 : 0.20
        Class 3 : 0.47
    Fit for 3 latent classes
    Estimation method             : 1-step
    Number of observations        : 150
    Number of latent classes      : 3
    Number of estimated parameters: 23
    Log-likelihood (LL)           : -437.2717
    -2LL                          : 874.5433
    Average LL               

In [ ]:
pd.crosstab(df['iris_flower_type'], df['mixed_pred'])

mixed_pred,0,1,2
iris_flower_type,,,
setosa,50,0,0
versicolor,0,29,21
virginica,0,1,49


In [ ]:
rand_score(df['iris_flower_type'], df['mixed_pred'])

0.8464429530201343

<a name="nan"></a>
## Mixtures with Missing Values

StepMix provides support for missing values. The different [model options](https://stepmix.readthedocs.io/en/latest/index.html) with a ```_nan``` suffix support missing values natively using full information maximum likelihood. Missing values should be encoded as ```pd.NA``` or ```np.NaN```.

In this section, we repeat the Gaussian mixture experiment, but with missing values. We will replace 20% of measurements with `NaNs`.

In [ ]:
continuous_data_nan = continuous_data.copy()

# Replace 20% of values with missing values
for i, c in enumerate(continuous_data_nan.columns):
  continuous_data_nan[c] = continuous_data_nan[c].sample(frac=.8, random_state=42*i)

continuous_data_nan

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,NaN,1.4,NaN
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,NaN
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,NaN,3.0,NaN,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,NaN,2.0
148,6.2,3.4,5.4,2.3


In [ ]:
# Gaussian mixture model. Allow missing values
model = StepMix(n_components=3, measurement="continuous_nan", verbose=1, random_state=123)

# Fit to data
model.fit(continuous_data_nan)

# Save class membership predictions to df
df['continuous_pred_nan'] = model.predict(continuous_data_nan)

Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:00<00:00,  7.64it/s, max_LL=-267, max_avg_LL=-1.78]

MODEL REPORT
    Measurement model parameters


          model_name                    gaussian_diag_nan                
          class_no                                      0       1       2
          param       variable                                           
          covariances petal length (cm)            0.0324  0.1959  0.3646
                      petal width (cm)             0.0113  0.0268  0.1045
                      sepal length (cm)            0.1217  0.2021  0.2963
                      sepal width (cm)             0.1185  0.0894  0.0987
          means       petal length (cm)            1.4591  4.1131  5.4208
                      petal width (cm)             0.2419  1.2665  1.9219
                      sepal length (cm)            4.9872  5.7368  6.5665
                      sepal width (cm)             3.4327  2.6527  3.0039
    Class weights
        Class 1 : 0.34
        Class 2 : 0.26
        Class 3 : 0.40
    Fit for 3 latent classes
    Estimation method             : 1-step
    Number of observations 

In [ ]:
pd.crosstab(df['iris_flower_type'], df['continuous_pred_nan'])

continuous_pred_nan,0,1,2
iris_flower_type,,,
setosa,50,0,0
versicolor,1,36,13
virginica,0,2,48


In [ ]:
rand_score(df['iris_flower_type'], df['continuous_pred_nan'])

0.8783892617449665